In [1]:
### NUMBER OF ACTIVE GENETIC DISORDERS


# We launch the config file with the installation of the R libraries
source("config.R")


# INPUTS : "en_product1.xml"; "en_product3_156.xml".


# OUTPUTS : "number_of_active_genetic_disorders"; "number_of_non_active_genetic_disorders";
# percentage_of_active_genetic_disorders"; "percentage_of_non_active_genetic_disorders".


  There is a binary version available but the source version is later:
       binary  source needs_compilation
rlist 0.4.6.1 0.4.6.2             FALSE



installing the source package 'rlist'





  There is a binary version available but the source version is later:
           binary source needs_compilation
data.table 1.14.0 1.14.4              TRUE

  Binaries will be installed
package 'data.table' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages


also installing the dependencies 'cli', 'rlang', 'vctrs'





  There are binary versions available but the source versions are later:
      binary source needs_compilation
cli    2.5.0  3.4.1              TRUE
rlang 0.4.11  1.0.6              TRUE
vctrs  0.3.8  0.4.2              TRUE
dplyr  1.0.6 1.0.10              TRUE

  Binaries will be installed
package 'cli' successfully unpacked and MD5 sums checked
package 'rlang' successfully unpacked and MD5 sums checked
package 'vctrs' successfully unpacked and MD5 sums checked
package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages
package 'XML' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages


Warning message:
"package 'methods' is not available (for R version 3.6.1)"
Warning message:
"package 'methods' is a base package, and should not be updated"


package 'xlsx' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages

  There is a binary version available but the source version is later:
         binary source needs_compilation
jsonlite  1.7.2  1.8.2              TRUE

  Binaries will be installed
package 'jsonlite' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'jsonlite'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\ohongnat\Anaconda3\Lib\R\library\00LOCK\jsonlite\libs\x64\jsonlite.dll vers C:\Users\ohongnat\Anaconda3\Lib\R\library\jsonlite\libs\x64\jsonlite.dll: Permission denied"
Warning message:
"restored 'jsonlite'"



The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages

  There is a binary version available but the source version is later:
        binary source needs_compilation
writexl  1.4.0  1.4.1              TRUE

  Binaries will be installed
package 'writexl' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpOwBuj9\downloaded_packages

  There is a binary version available but the source version is later:
        binary source needs_compilation
stringr  1.4.0  1.4.1             FALSE



installing the source package 'stringr'


Warning message:
"package 'data.table' was built under R version 3.6.3"
Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'XML' was built under R version 3.6.3"
Warning message:
"package 'xlsx' was built under R version 3.6.3"
Warning message:
"package 'writexl' was built under R version 3.6.3"


In [2]:
### Later we will need to get the number of total disorders for the calculations

# We open xml "product1_en_cross_jdbor_evo" that contains these datas
product1=xmlParse("en_product1.xml")
product1=xmlToList(product1)

# We remove the last element of the list, which is the numeric count of all elements of the list
product1=product1$DisorderList
product1=product1[-length(product1)]

### Our list for working
indice_list=1
product=list()
for (i in 1:length(product1))
{
  product[[indice_list]]=product1[[i]]
  indice_list=indice_list+1
}
# Each element in the list is a disorder

In [3]:
# We only select the active clinical entities:
# Head of classification (flag=128), Historical entities (flag=512) & On-line (flag=1)
indice_list=1
all_active_clinical_entities=list()
for (i in 1:length(product))
  if (product[[i]]$DisorderFlagList$DisorderFlag$Value=='1' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='128' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='512')
  {
    all_active_clinical_entities[[indice_list]]=product[[i]]
    indice_list=indice_list+1
  }

In [4]:
### Number of disorders
indice_list=1
disorders=list()
for (i in 1:length(all_active_clinical_entities))
  if (all_active_clinical_entities[[i]]$DisorderGroup$.attrs=='36547')
  {
    disorders[[indice_list]]=all_active_clinical_entities[[i]]
    indice_list=indice_list+1
  }
number_disorders=length(disorders)
print(paste0("The number of disorders is ", number_disorders))

[1] "The number of disorders is 6227"


In [5]:
# We can now work on the genetic datas

# We open the xml "product3_156_en_jdbor_evo.xml" that contains the genetic disorders
xmlfiles = list.files(pattern='product3_156')
xmldoc=xmlParse(file=xmlfiles)
xml_156=xmlToList(xmldoc)

# We get the Classification information
prod_genetic=as.list(unlist(xml_156$ClassificationList))
# Each element in the list is a disorder

In [6]:
# We get the OrphaCode and the DisorderType for each disorder
disordertype = list()
orpha = list()
indice_orpha = 1
indice_disordertype = 1
for (i in 1:length(prod_genetic))
  if (grepl("Disorder.OrphaCode", names(prod_genetic[i]))) {
    orpha[[indice_orpha]] = data.frame(orpha = prod_genetic[i])
    names(orpha[[indice_orpha]]) = c('orpha')
    indice_orpha = indice_orpha + 1
  } else if (grepl("DisorderType..attrs.id", names(prod_genetic[i]))) {
    disordertype[[indice_disordertype]] = data.frame(disordertype = prod_genetic[i])
    names(disordertype[[indice_disordertype]]) = c('disordertype')
    indice_disordertype = indice_disordertype + 1
  } 

disordertype = rbindlist(disordertype)
orpha = rbindlist(orpha)

In [7]:
# We only keep Disorders
disorders = cbind.data.frame(orpha, disordertype)
disorders = filter(disorders, disordertype == "21429" |
                     disordertype == "21401" |
                     disordertype == "21415" |
                     disordertype == "21394" |
                     disordertype == "21422" |
                     disordertype == "21408")

disorders = filter(disorders, !duplicated(disorders))

In [8]:
# Then we finaly get the number of genetic disorders
number_of_active_genetic_disorders=nrow(disorders)
print(paste0("The number of genetic disorders is ", number_of_active_genetic_disorders))

[1] 4521

[1] "The number of genetic disorders is 4521"


In [9]:
# And the number of non genetic disorders
number_of_non_active_genetic_disorders=number_disorders - number_of_active_genetic_disorders
print(paste0("The number of non-genetic disorders is ", number_of_non_active_genetic_disorders))

[1] 1706

[1] "The number of non-genetic disorders is 1706"


In [10]:
# Percentage of genetic disorders
percentage_of_active_genetic_disorders=number_of_active_genetic_disorders/number_disorders*100
print(paste0("The percentage of genetic disorders is ", percentage_of_active_genetic_disorders, "%"))

[1] 72.60318

[1] "The percentage of genetic disorders is 72.6031797013008%"


In [11]:
# Percentage of non genetic disorders
percentage_of_non_active_genetic_disorders=number_of_non_active_genetic_disorders/number_disorders*100
print(paste0("The percentage of non-genetic disorders is ", percentage_of_non_active_genetic_disorders, "%"))

[1] 27.39682

[1] "The percentage of non-genetic disorders is 27.3968202986992%"
